In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
from ase.io import read as ase_read
import umap

from dadapy.feature_weighting import FeatureWeighting

In [ ]:
import pandas as pd

# Load

In [ ]:
data_dir = pathlib.Path('../data').resolve()

In [ ]:
# Get ase.Atoms objects for each liquid configuration
liquid_frames = ase_read(data_dir.joinpath("ice_in_water_data/dataset_1000_eVAng.xyz"), index=':')
n_atoms = np.sum(np.asarray([len(frame) for frame in liquid_frames], dtype=np.int16))

In [ ]:
n_atoms

In [ ]:
atomic_soap = np.load(data_dir.joinpath("ice_in_water_data/singleatom_soap_rcut6_nmax6_lmax6_sigma03.npy"))

# The file format of the input file the descriptors are calculated from is 54 solid, 1000 liquid
# So we can just get the liquid configurations by getting the number of atoms n_atoms in the liquid configurations
# From the end of the decriptor matrix
liquid_atomic_soap = atomic_soap[-n_atoms:, :].copy()

In [ ]:
atomic_acsf = np.asarray(np.load(data_dir.joinpath("ice_in_water_data/singleatom_acsf_rcut6_gridsearch_bohr_lambda.npy")), dtype=np.float32)
liquid_atomic_acsf = atomic_acsf[-n_atoms:, :].copy()

In [ ]:
# descriptors = [atomic_soap, liquid_atomic_soap, atomic_acsf, liquid_atomic_acsf]
# for desc in descriptors:
#     desc /= np.linalg.norm(desc, axis=-1)[:, np.newaxis]
# atomic_soap, liquid_atomic_soap, atomic_acsf, liquid_atomic_acsf = descriptors

In [ ]:
soap_cut = liquid_atomic_soap[::500]
acsf_cut = liquid_atomic_acsf[::500]
acsf_cut.shape

In [ ]:
soap_cut

In [ ]:
np.isnan(soap_cut).any()

# UMAP

In [ ]:
reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric='euclidean')
embedding = reducer.fit_transform(soap_cut)

In [ ]:
umap=embedding

In [ ]:
plt.plot(umap[:,0], umap[:,1], 'o')

### There are some 0 columns in the ACSFs. drop them

In [ ]:
zero_columns = np.where(np.all(acsf_cut == 0, axis=0))
acsf_cut = np.delete(acsf_cut, zero_columns, axis=1)

# DII

In [ ]:
n_epochs = 70  # number of training epochs
l1s=[0 ,0.1, 10, 1000, 100000]

f = FeatureWeighting(coordinates=acsf_cut, verbose=True)
f_target = FeatureWeighting(coordinates=umap)

(
    num_nonzero_features,
    l1_penalties_opt_per_nfeatures,
    dii_opt_per_nfeatures,
    weights_opt_per_nfeatures,
) = f.return_lasso_optimization_dii_search(
    target_data=f_target,
    initial_weights=None,  # (default) set automatically
    n_epochs=n_epochs,
    l1_penalties=l1s,
    learning_rate=None,  # (default) set automatically
    refine=False,  # only 10 values of the L1 strength are tested
    plotlasso=True,  # automatically show DII vs number of non-zero features
)

In [ ]:
n_epochs = 70  # number of training epochs
l1s=[10000000, 100000000, 1000000000, 10000000000]

f = FeatureWeighting(coordinates=acsf_cut, verbose=True)
f_target = FeatureWeighting(coordinates=umap)

(
    num_nonzero_features,
    l1_penalties_opt_per_nfeatures,
    dii_opt_per_nfeatures,
    weights_opt_per_nfeatures,
) = f.return_lasso_optimization_dii_search(
    target_data=f_target,
    initial_weights=None,  # (default) set automatically
    n_epochs=n_epochs,
    l1_penalties=l1s,
    learning_rate=None,  # (default) set automatically
    refine=False,  # only 10 values of the L1 strength are tested
    plotlasso=True,  # automatically show DII vs number of non-zero features
)

# Backward greedy

In [ ]:
n_epochs = 70  # number of training epochs

final_imbs, final_weights = f.return_backward_greedy_dii_elimination(
    target_data=f_target,
    initial_weights=None,  # set automatically (default)
    n_epochs=n_epochs,
    learning_rate=None,  # set automatically (default)
)

In [ ]:
np.savetxt("final_imbs.csv", final_imbs)
np.savetxt("final_weights.csv", final_weights)

In [ ]:
plt.plot(np.arange(len(final_imbs), 0, -1),final_imbs,  ".-")